In [29]:
from pyspark.sql import SparkSession, functions as f
from pyspark.sql.functions import col
from itertools import combinations

In [30]:
# spark = SparkSession.builder.appName('walmart-data').master('spark://spark-master:7077').getOrCreate()
spark = SparkSession.builder.appName('walmart-data').master('local').getOrCreate()

In [31]:
product_schema = 'product_id INT, product_name STRING, aisle_id INT, department_id INT'

order_product_schema = 'order_id INT, product_id INT, add_to_cart_order INT, reordered INT'

In [32]:
df_products = spark.read.option("header", True).schema(product_schema).csv('./datasets/products.csv').select(
    'product_id')  # Items set
df_products.show(5)

+----------+
|product_id|
+----------+
|         1|
|         2|
|         3|
|         4|
|         5|
+----------+


In [33]:
products_qty = df_products.count()
products_qty

49688

In [38]:
def generate_combinations(row):
    print(row)
    id_list = row[0]
    return combinations(id_list, row[1])

# df_products.select("product_id").rdd.flatMap(lambda row: combinations(row, 2)).toDF(['id1', 'id2']).show()
df_products[
# result_df = combinations_rdd.toDF()

Column<'product_id[0][0]'>

In [ ]:
df_order_products = spark.read.option("header", True).schema(order_product_schema).csv(
    './datasets/order_products__prior.csv').drop("add_to_cart_order", "reordered").dropDuplicates()
df_order_products.show(5)

In [ ]:
df_transactions = (df_order_products
                   .groupBy('order_id')
                   .agg(f.collect_set('product_id').alias('items'))
                   .withColumnRenamed('order_id', 'transaction_id'))
df_transactions.show(5, truncate=False)

In [ ]:
transactions_qty = df_transactions.count()
transactions_qty

In [ ]:
min_support = 0.025

In [ ]:
df_frequent_items = (df_order_products.
                     groupBy('product_id')
                     .count()
                     .withColumn('count', f.col('count') / transactions_qty)
                     .withColumnRenamed('count', 'support')
                     .orderBy(f.col('product_id').asc())
                     .filter(f.col('support') >= min_support))
df_frequent_items.show(10)